# Build Tempest Sources for Version 2A(Alt)

In [24]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))

## Prepare the Code

Clone the historicalsources source dump.

In [13]:
!rm -rf tempest_2A
!git clone https://github.com/historicalsource/tempest tempest_2A

Cloning into 'tempest_2A'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 45 (delta 8), reused 45 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 174.01 KiB | 1001.00 KiB/s, done.
Resolving deltas: 100% (8/8), done.


Apply our patch that allows it to assemble and link correctly. This patch removes the dependency (which our old version of MAC65 can't handle) on correctly assembling globals with an offset.

In [14]:
!cd tempest_2A && git apply ../0001-Fix-byte-relocation-errors.patch 

../0001-Fix-byte-relocation-errors.patch:76: trailing whitespace.



Create an rk05 disk of our modified sources.

In [15]:
from rt11_utils import create_disk_from_files
create_disk_from_files("tempest_2A/", "tempest_2A.rk05", ["MAC"])

Copy it over to the `atari_build` directory so that our `pdp11` environment will pick it up when we run it in a few minutes. 

In [16]:
!cp tempest_2A.rk05 ../atari_build/tempest_original.rk05

## Let's Build It

For a full overview of the build process (plus tips and tricks), take a look at [the `atari_build` folder](../atari_build). Otherwise just have a go at the steps below.

First, make sure you have `simh` installed:
```
sudo apt install simh
```

Then fire up a pdp11 from within the `atari_build` folder. If you have `gnome-terminal` installed, you can just run the cell below:

In [17]:
!cd ../atari_build && gnome-terminal -e "pdp11 tempest.ini"

# Option “-e” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Failed to use specified server: GDBus.Error:org.freedesktop.DBus.Error.ServiceUnknown: The name :1.94 was not provided by any .service files
# Falling back to default server.


Before you do anything type the following command, this will ensure that backspace works for you:
```
SET TT SCOPE
```

Below are the commands for building version 2A(alt) of Tempest. You can copy this into the terminal with PDP-11 running. The files ending with `2` are the '2A' versions. When building version 1 we use different files.

```
R MAC65
RK1:ALWELG=ALWELG
RK1:ALSCO2=ALSCO2
RK1:ALDIS2=ALDIS2
RK1:ALEXEC=ALEXEC
RK1:ALSOUN=ALSOUN
RK1:ALVROM=ALVROM
^C
```

To run the second batch press `Ctrl-C` to 'close' the assembler first, then paste:

```
R MAC65
RK1:ALCOIN=ALCOIN
RK1:ALLANG=ALLANG
RK1:ALHAR2=ALHAR2
RK1:ALTES2=ALTES2
RK1:ALEARO=ALEARO
RK1:ALVGUT=ALVGUT
^C
```


The command to link our object files is:
```
R LINKM
RK1:ALEXEC/L,ALEXEC/A=RK1:ALWELG,ALSCO2,ALDIS2,ALEXEC,ALSOUN,ALVROM/C
ALCOIN,ALLANG,ALHAR2,ALTES2,ALEARO,ALVGUT
^C
```



When we're done building, we can close the terminal. Now we copy the rk05 disk, which now contains our object files and final binary, back over here.

In [18]:
!cp ../atari_build/tempest_original.rk05 tempest_2A_objects.rk05 

## Extract our Build and Compare it with the Original

Extract the files from our rk05 disk.

In [19]:
from rt11_utils import extract_files_from_disk
extract_files_from_disk("tempest_2A_objects.rk05", "tempest_2A_mod_object_files")

Compare the `ALEXEC.LDA` we built using our patched sources with the original one from the source dump.

In [21]:
!diff -y -W 150 <(dd if=tempest_modified/ALEXEC.LDA|xxd) <(dd if=tempest_2A_mod_object_files/ALEXEC.LDA|xxd) |grep '  |	'

77+0 records in
77+0 records out
39424 bytes (39 kB, 38 KiB) copied, 6.1706e-05 s, 639 MB/s
77+0 records in
77+0 records out
39424 bytes (39 kB, 38 KiB) copied, 6.6681e-05 s, 591 MB/s


Voila! They match! But let's play this version that we built. First we extract the raw bytes from our object file `ALEXEC.LDA`:

## Generate the ROM Files from our Build Binary

In [28]:
from rt11_utils import parse_lda_file
output_bytes = parse_lda_file("tempest_2A_mod_object_files/ALEXEC.LDA")

Now we can use these extracted bytes to fill the required ROM files. First we fill the ROMs that contain all the game code:

In [29]:
import os
OUT_DIR="tempest_2A_roms/tempest"
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

open(f"{OUT_DIR}/136002-138.np3",'wb').write(bytes(output_bytes[0x3000:0x4000]))
# Notice we have the addresses in reverse order, e.g. 136002-237 is written to 0xD000 rather
# than 0x9000.
open(f"{OUT_DIR}/136002-237.p1",'wb').write(bytes(output_bytes[0xD000:0xE000]))
open(f"{OUT_DIR}/136002-136.lm1",'wb').write(bytes(output_bytes[0xC000:0xD000]))
open(f"{OUT_DIR}/136002-235.j1",'wb').write(bytes(output_bytes[0xB000:0xC000]))
open(f"{OUT_DIR}/136002-134.f1",'wb').write(bytes(output_bytes[0xA000:0xB000]))
open(f"{OUT_DIR}/136002-133.d1",'wb').write(bytes(output_bytes[0x9000:0xA000]))


4096

Next we fill the ROMS that will contain the 'mathbox' code. This is like a help library used by the game for math calculations:

In [30]:
mathbox_bytes = open("tempest/MBOX.SAV",'rb').read()
open(f"{OUT_DIR}/136002-126.a1",'wb').write(mathbox_bytes[0x8400:0x8420])
# 136002-127.e1 contains the low 4 bits of each byte in 0x6A00:0x6B00 in MBOX.SAV
open(f"{OUT_DIR}/136002-127.e1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6A00:0x6B00]]))
# 136002-128.f1 contains the high 4 bits (right-shifted) of each byte in 0x6A00:0x6B00 in MBOX.SAV
open(f"{OUT_DIR}/136002-128.f1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6A00:0x6B00]]))

# 136002-129.h1 contains the low 4 bits of each byte in 0x6900:0x6A00 in MBOX.SAV
open(f"{OUT_DIR}/136002-129.h1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6900:0x6A00]]))
# 136002-130.j1 contains the high 4 bits (right-shifted) of each byte in 0x6900:0x6A00 in MBOX.SAV
open(f"{OUT_DIR}/136002-130.j1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6900:0x6A00]]))

# 136002-131.k1 contains the low 4 bits of each byte in 0x6800:0x6900 in MBOX.SAV
open(f"{OUT_DIR}/136002-131.k1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6800:0x6900]]))
# 136002-132.l1 contains the high 4 bits (right-shifted) of each byte in 0x6800:0x6900 in MBOX.SAV
open(f"{OUT_DIR}/136002-132.l1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6800:0x6900]]))


256

Finally we fill the ROM that contains the code used to generate vectors in the game. Tempest is a vector rather than pixel-based game after all:

In [31]:
vecgen_bytes = open("tempest/STATE2.SAV",'rb').read()
open(f"{OUT_DIR}/136002-125.d7",'wb').write(vecgen_bytes[0x1000:0x1100])

256

## Play the Game We Built With Our Own Hands

And finally we can run the game!

In [32]:
!mame -window -rompath ./tempest_2A_roms/ tempest

Unable to find the BGFX path bgfx, please install it or fix the bgfx_path setting to use the BGFX renderer.
Average speed: 99.86% (2 seconds)
